## Stochastic MPC using State Estimation and Terminal Cost

# Specify perturbation

In [ ]:
import numpy as np

# np.random.seed(42)
nx = 25  # number of grid blocks
Ne = 10 # number of perturbation
amean = 5.0
bmean = 9.0
a_std = amean/3
b_std = bmean/3
amin = 3.5
amax = 6.5
bmin = 7.5
bmax = 10.5

# aw.shape

In [ ]:
import gstools as gs
import scipy.stats



if True:
    aw = np.zeros((nx, Ne))
    bw = np.zeros((nx, Ne))
    
    for i in range(Ne):
        model = gs.Gaussian(dim=1, var=0.5, len_scale=5)
        srf = gs.SRF(model)
        x = range(25)
        srf(x, mesh_type='structured')
        fieldcdf = scipy.stats.norm.cdf(srf.field, 0, 1)
        a = (amin - amean) / a_std
        b = (amax - amean) / a_std
        var = scipy.stats.truncnorm.ppf(fieldcdf, a, b)
        _aw = var*a_std + amean

        model = gs.Gaussian(dim=1, var=0.5, len_scale=5)
        srf = gs.SRF(model)
        x = range(25)
        srf(x, mesh_type='structured')
        fieldcdf = scipy.stats.norm.cdf(srf.field, 0, 1)
        a = (bmin - bmean) / b_std
        b = (bmax - bmean) / b_std
        var = scipy.stats.truncnorm.ppf(fieldcdf, a, b)
        _bw = var*b_std + bmean

        aw[:,i] = _aw
        bw[:,i] = _bw

    np.save("./data/aw.npy", aw)
    np.save("./data/bw.npy", bw)
    
if False:
    model = gs.Gaussian(dim=1, var=0.5, len_scale=5)
    srf = gs.SRF(model)
    x = range(25)
    srf(x, mesh_type='structured')
    fieldcdf = scipy.stats.norm.cdf(srf.field, 0, 1)
    a = (amin - amean) / a_std
    b = (amax - amean) / a_std
    var = scipy.stats.truncnorm.ppf(fieldcdf, a, b)
    _aw = var*a_std + amean

    model = gs.Gaussian(dim=1, var=0.5, len_scale=5)
    srf = gs.SRF(model)
    x = range(25)
    srf(x, mesh_type='structured')
    fieldcdf = scipy.stats.norm.cdf(srf.field, 0, 1)
    a = (bmin - bmean) / b_std
    b = (bmax - bmean) / b_std
    var = scipy.stats.truncnorm.ppf(fieldcdf, a, b)
    _bw = var*b_std + bmean

    a_true = _aw*1
    b_true = _bw*1
    
    np.save("./data/a_true.npy", a_true)
    np.save("./data/b_true.npy", b_true)
    
    
import matplotlib.pyplot as plt

aw = np.load("./data/aw.npy")
bw = np.load("./data/bw.npy")

a_true = np.load("./data/a_true.npy")
b_true = np.load("./data/b_true.npy")

fig, ax = plt.subplots(2, 1, figsize=(8,6), sharex=True)
ax[0].plot(aw, color='gray', alpha=0.5)
ax[0].plot(a_true, color='red', linewidth=2)
ax[1].plot(bw, color='gray', alpha=0.5)
ax[1].plot(b_true, color='red', linewidth=2)
ax[0].plot([], color='gray', alpha=0.5, label="a prior")
ax[0].plot([], color='red', linewidth=2, label="true a")
ax[1].plot([], color='gray', alpha=0.5, label="b prior")
ax[1].plot([], color='red', linewidth=2, label="true b")
ax[0].legend()
ax[1].legend()
plt.show()

## Find the solution to the stochastic OCP solution 

In [ ]:

import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from setup_sbl_te import setup_sbl_ocp, BLParamsSmpcShort
from solver_socp_te import SolverOcp

def make_ensemble_propagator(f_discrete, nx, Ne):
    Sw_flat = ca.SX.sym("Sw_flat", nx*Ne)
    u_sym = ca.SX.sym("u")
    aw_sym = ca.SX.sym("aw", nx, Ne)
    bw_sym = ca.SX.sym("bw", nx, Ne)
    outs = []
    for i in range(Ne):
        Si = Sw_flat[i*nx:(i+1)*nx]
        outs.append(f_discrete(Si, u_sym, aw_sym[:,i], bw_sym[:,i]))
        # outs.append(f_discrete(Si, u_sym, aw_sym[i], bw_sym[i]))
    Sw_next_flat = ca.vertcat(*outs)
    return ca.Function("propagate_ensemble", [Sw_flat, u_sym, aw_sym, bw_sym], [Sw_next_flat])


if True:
    qocp = np.load("./data/stochastic_socp_qt.npy")[0]

    n_mpc_steps = 1
    N_mpc = 100

    params_mpc = BLParamsSmpcShort(N_mpc = N_mpc, 
                                Ne = Ne,
                                nx = nx,
                                aw = aw,
                                bw = bw)

    nx = params_mpc.nx

    qt = np.array([1.0]*params_mpc.N_mpc)  # initial guess for control

    u_traj = qt*1

    _Sw0 = [1.0]+[0.2]*(params_mpc.nx-1)
    Sw0 = np.array(_Sw0*params_mpc.Ne)  # initial state
    Sw_mpc = Sw0.copy()

    qmpc = [1.0]

    for mpc_step in tqdm(range(n_mpc_steps)):
        _qocp = qocp[(N_mpc+mpc_step):]
        
        ocp = setup_sbl_ocp(params_mpc, 
                            qmpc=qmpc,
                            qocp=_qocp)
        
        aw = params_mpc.aw
        bw = params_mpc.bw
        
        if mpc_step == 0:
            f_discrete = ocp.bl.simulate_at_k
            f = make_ensemble_propagator(f_discrete, nx, Ne)
        
        solver = SolverOcp(ocp, itk=mpc_step)
        solver.set_initial_guess(Sw0, qt)
        x_traj, u_traj = solver.solve(Sw0)

    qocp = np.insert(u_traj[:,0], 0, 1.0)[:-1]
        

    # Compute the cost from the OCP solutions


    Sw_traj = []
    cost_traj = []
    for k, q in enumerate(qocp):
        
        costs = []
        for i in range(params_mpc.Ne):
            Sw = Sw_mpc[i*nx:(i+1)*nx]
            cost = -ocp.bl._stage_cost(Sw, q)*(0.99**k)
            # cost = -ocp.bl._stage_cost(Sw, q)*(1.0**k)
            # costs.append(cost.full()[0,0])
            costs.append(cost)
            
        Sw_mpc = f(Sw_mpc, q, aw, bw)
        Sw_traj.append(Sw_mpc)
        cost_traj.append(costs)
        

    qocp = np.array(qocp)
    Sw_traj = np.array(Sw_traj)  # (n_mpc_steps, nx*Ne)
    cost_traj = np.array(cost_traj)  # (n_mpc_steps, Ne)

    np.save("./data/socp_qocp.npy", qocp)
    np.save("./data/socp_Sw_traj.npy", Sw_traj)
    np.save("./data/socp_cost_traj.npy", cost_traj)
        

In [ ]:

fig, ax = plt.subplots()
qocp = np.load("./data/socp_qocp.npy")
ax.stairs(range(len(qocp)-1), qocp, label="Stochastic OCP", color="blue", orientation="horizontal", baseline=None)
ax.legend()
ax.set_ylabel("Injection rate")
ax.set_xlabel("Time step")
ax.set_ylim([0.78, 1.22])
ax.grid()
plt.show()

ocp_cashflow = np.load("./data/socp_cost_traj.npy")[:,:,0,0]

fig, ax = plt.subplots()
ax.plot(np.cumsum(ocp_cashflow, axis=0), color='blue')
ax.plot([], color='blue', label="Stochastic OCP")
ax.set_ylabel("NPV")
ax.set_xlabel("Time step")
ax.legend(loc="center right")

ax1 = ax.twinx()
ax1.plot(ocp_cashflow, color="blue", linestyle="dashed")
ax1.set_ylabel("Cashflow")

ax1.grid()


## Compute terminal cost using optimal control solution to the stochastic OCP

In [ ]:

import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from setup_sbl_te import setup_sbl_ocp, BLParamsSmpcShort
from solver_socp_te import SolverOcp

def make_ensemble_propagator(f_discrete, nx, Ne):
    Sw_flat = ca.SX.sym("Sw_flat", nx*Ne)
    u_sym = ca.SX.sym("u")
    aw_sym = ca.SX.sym("aw", nx, Ne)
    bw_sym = ca.SX.sym("bw", nx, Ne)
    outs = []
    for i in range(Ne):
        Si = Sw_flat[i*nx:(i+1)*nx]
        outs.append(f_discrete(Si, u_sym, aw_sym[:,i], bw_sym[:,i]))
        # outs.append(f_discrete(Si, u_sym, aw_sym[i], bw_sym[i]))
    Sw_next_flat = ca.vertcat(*outs)
    return ca.Function("propagate_ensemble", [Sw_flat, u_sym, aw_sym, bw_sym], [Sw_next_flat])

if False:
    qocp = np.load("./data/socp_qocp.npy")

    n_mpc_steps = 100
    N_mpc = 10

    params_mpc = BLParamsSmpcShort(N_mpc = N_mpc, 
                                Ne = Ne,
                                nx = nx,
                                aw = aw,
                                bw = bw)

    nx = params_mpc.nx

    qt = np.array([1.0]*params_mpc.N_mpc)  # initial guess for control

    u_traj = qt*1

    _Sw0 = [1.0]+[0.2]*(params_mpc.nx-1)
    Sw0 = np.array(_Sw0*params_mpc.Ne)  # initial state
    Sw_mpc = Sw0.copy()

    qmpc = [1.0]
    Sw_traj = []

    cost_traj = []


    for mpc_step in tqdm(range(n_mpc_steps)):
        _qocp = qocp[(N_mpc+mpc_step):]
        
        ocp = setup_sbl_ocp(params_mpc, 
                            qmpc=qmpc,
                            qocp=_qocp)
        
        aw = params_mpc.aw
        bw = params_mpc.bw
        
        # if mpc_step == 0:
        f_discrete = ocp.bl.simulate_at_k
        f = make_ensemble_propagator(f_discrete, nx, Ne)
        
        
        costs = []
        for i in range(params_mpc.Ne):
            Sw = Sw_mpc[i*nx:(i+1)*nx]
            cost = -ocp.bl._stage_cost(Sw, qmpc[-1])*(0.99**mpc_step)
            # cost = -ocp.bl._stage_cost(Sw, qmpc[-1])*(1.0**mpc_step)
            costs.append(cost)
        cost_traj.append(costs)
        
        Sw_mpc = f(Sw_mpc, qmpc[-1], aw, bw)
        Sw_traj.append(Sw_mpc)
        
        solver = SolverOcp(ocp, itk=mpc_step)
        # warm start
        if mpc_step == 0:
            solver.set_initial_guess(Sw0, qt)
        else:
            _qt = np.insert(u_traj[1:,0], N_mpc-1, u_traj[-1,0])
            solver.set_initial_guess(Sw0, _qt)
        x_traj, u_traj = solver.solve(Sw0)
        
        qmpc.append(u_traj[0,0]*1)
        
        
        
        

    qmpc = np.array(qmpc)
    Sw_traj = np.array(Sw_traj)  # (n_mpc_steps, nx*Ne)
    cost_traj = np.array(cost_traj)  # (n_mpc_steps, Ne)

    np.save("./data/smpc_qmpc.npy", qmpc)
    np.save("./data/smpc_Sw_traj.npy", Sw_traj)
    np.save("./data/smpc_cost_traj.npy", cost_traj)
        

In [ ]:
fig, ax = plt.subplots()
qocp = np.load("./data/socp_qocp.npy")
qmpc = np.load("./data/smpc_qmpc.npy")
ax.plot(qocp, label="Stochastic OCP", color="blue")
plt.plot(qmpc[:-1], label="MPC control", color="orange")
ax.legend()
ax.set_ylabel("Injection rate")
ax.set_xlabel("Time step")
ax.set_ylim([0.78, 1.22])
ax.grid()
plt.show()

socp_cost = np.load("./data/socp_cost_traj.npy")[:,:,0,0]
smpc_cost = np.load("./data/smpc_cost_traj.npy")[:,:,0,0]

fig, ax = plt.subplots()
ax.plot(np.cumsum(socp_cost, axis=0), color='blue')
ax.plot(np.cumsum(smpc_cost, axis=0), color='orange')
ax.plot([], color='blue', label="Stochastic OCP")
ax.plot([], color='orange', label="Stochastic MPC")
ax.set_ylabel("NPV")
ax.set_xlabel("Time step")
ax.legend(loc="center right")

ax1 = ax.twinx()
ax1.plot(socp_cost, color="blue", linestyle="dashed")
ax1.plot(smpc_cost, color="orange", linestyle="dashed")
ax1.set_ylabel("Cashflow")

ax1.grid()


npv_socp = np.cumsum(socp_cost, axis=0)[-1,:].mean()
npv_smpc = np.cumsum(smpc_cost, axis=0)[-1,:].mean()

npv_socp, npv_smpc

## Compute the cost using the optimal control on the true model, then update the parameter

In [ ]:


import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from setup_sbl_te import setup_sbl_ocp, BLParamsSmpcShort, BLParamsTrue
from solver_socp_te import SolverOcp

from kalman_filter import ESMDA

def make_ensemble_propagator(f_discrete, nx, Ne):
    Sw_flat = ca.SX.sym("Sw_flat", nx*Ne)
    u_sym = ca.SX.sym("u")
    aw_sym = ca.SX.sym("aw", nx, Ne)
    bw_sym = ca.SX.sym("bw", nx, Ne)
    outs = []
    for i in range(Ne):
        Si = Sw_flat[i*nx:(i+1)*nx]
        outs.append(f_discrete(Si, u_sym, aw_sym[:,i], bw_sym[:,i]))
        # outs.append(f_discrete(Si, u_sym, aw_sym[i], bw_sym[i]))
    Sw_next_flat = ca.vertcat(*outs)
    return ca.Function("propagate_ensemble", [Sw_flat, u_sym, aw_sym, bw_sym], [Sw_next_flat])

if True:
    qocp = np.load("./data/socp_qocp.npy")
    aw = np.load("./data/aw.npy")
    bw = np.load("./data/bw.npy")

    a_true = np.load("./data/a_true.npy")
    b_true = np.load("./data/b_true.npy")


    n_mpc_steps = 100
    N_mpc = 5

    params_mpc = BLParamsSmpcShort(N_mpc = N_mpc, 
                                Ne = Ne,
                                nx = nx,
                                aw = aw,
                                bw = bw)

    params_mpc_true = BLParamsTrue(N_mpc = N_mpc, 
                                Ne = 1,
                                nx = nx,
                                aw = a_true[:,np.newaxis],
                                bw = b_true[:,np.newaxis])

    esmda = ESMDA(m=aw, g_func=None, g_obs=np.array([0.01]), alphas=[], cd=[0.01])

    qt = np.array([1.0]*params_mpc.N_mpc)  # initial guess for control

    u_traj = qt*1

    _Sw0 = [1.0]+[0.2]*(params_mpc.nx-1)
    Sw0 = np.array(_Sw0*params_mpc.Ne)  # initial state
    Sw_mpc = Sw0.copy()
    Sw_true = np.array(_Sw0.copy())

    qmpc = [1.0]

    cost_traj = []

    Sw_traj = []
    Sw_true_traj = []

    aw_traj = []
    bw_traj = []
    
    n_obs_data_count = 0
    N_obs = 5 # number of data points to be used simultaneously in the parm estimation
    is_breakthrough = False
    g_meas = np.zeros((Ne, N_obs))
    g_obs = np.zeros((N_obs))
    
    for mpc_step in tqdm(range(n_mpc_steps)):
        _qocp = qocp[(N_mpc+mpc_step):]
        
        params_mpc = BLParamsSmpcShort(N_mpc = N_mpc, 
                                Ne = Ne,
                                nx = nx,
                                aw = aw,
                                bw = bw)
        
        ocp = setup_sbl_ocp(params_mpc, 
                            qmpc=qmpc,
                            qocp=_qocp)
        
        ocp_true = setup_sbl_ocp(params_mpc_true, 
                            qmpc=qmpc,
                            qocp=_qocp)
        
        aw = params_mpc.aw
        bw = params_mpc.bw
        
        a_true = params_mpc_true.aw
        b_true = params_mpc_true.bw
        
        if mpc_step == 0:
            f_discrete = ocp.bl.simulate_at_k
            f = make_ensemble_propagator(f_discrete, nx, Ne)
        
        # activate parameter estimation procedure here
        # only when water breakthrough is observed, i.e., saturation at production well > 0.5
        # if Sw_true[-1] > 0.5 for any i in range(Ne)
        
        # observed data is the saturation at the last grid cell after the breakthrough
        
        
        
        if Sw_true[-1] > 0.2:
            
            if not is_breakthrough:
                t_breakthrough = mpc_step*1
                is_breakthrough = True
            
            # collect ensemble response
            for i in range(Ne):
                Sw_last = Sw_mpc[(i+1)*nx-1]
                g_meas[i, n_obs_data_count] = Sw_last
                    
            # collect observed response
            g_obs[n_obs_data_count] = Sw_true[-1]
                    
            if n_obs_data_count < N_obs-1:
                n_obs_data_count += 1
                
            else:
                # perform parameter estimation update here
                    
                # g_meas = np.array([g_meas]).T    
                # g_obs = np.array(Sw_true[[-1]])
                
                NiterESMDA = 1 
                
                # for ii in range(NiterESMDA):
                # cd = np.array([0.0000001]*N_obs)
                # cd = np.array([0.000001]*N_obs)
                cd = np.array([0.0001]*N_obs)
                
                aw_post = np.exp(esmda.update(np.log(aw.T), g_meas, g_obs, 1, cd))
                bw_post = esmda.update(bw.T, g_meas, g_obs, 1, cd)
                
                aw = aw_post.T
                bw = bw_post.T
                
                aw_traj.append(aw)
                bw_traj.append(bw)
        
                # reset 
                g_meas = np.zeros((Ne, N_obs))
                g_obs = np.zeros((N_obs))
                n_obs_data_count = 0
        
        costs = []
        for i in range(params_mpc.Ne):
            Sw = Sw_mpc[i*nx:(i+1)*nx]
            cost = -ocp.bl._stage_cost(Sw, qmpc[-1])*(0.99**mpc_step)
            # cost = -ocp.bl._stage_cost(Sw, qmpc[-1])*(1.0**mpc_step)
            costs.append(cost)
        cost_traj.append(costs)
        
        # simulate the ensemble
        Sw_mpc = f(Sw_mpc, qmpc[-1], aw, bw).full()[:,0]

        # simulate the true model
        Sw_true = f_discrete(Sw_true, qmpc[-1], a_true[:,0], b_true[:,0]).full()[:,0]
        
        solver = SolverOcp(ocp, itk=mpc_step)
        
        # warm start
        if mpc_step == 0:
            solver.set_initial_guess(Sw0, qt)
        else:
            _qt = np.insert(u_traj[1:,0], N_mpc-1, u_traj[-1,0])
            solver.set_initial_guess(Sw0, _qt)
            
        x_traj, u_traj = solver.solve(Sw0)
        
        qmpc.append(u_traj[0,0]*1)
        
        Sw_traj.append(Sw_mpc)
        Sw_true_traj.append(Sw_true)
        
        
    cost_traj = np.array(cost_traj)
    
    np.save("./data/aw_traj.npy", aw_traj)
    np.save("./data/bw_traj.npy", bw_traj)

    np.save("./data/smpc_wse_qmpc.npy", qmpc) # wse = with state estimation
    np.save("./data/smpc_wse_Sw_traj.npy", Sw_traj)
    np.save("./data/smpc_wse_cost_traj.npy", cost_traj)

In [ ]:
fig, ax = plt.subplots()
qocp = np.load("./data/socp_qocp.npy")
# qmpc = np.load("./data/smpc_qmpc.npy")
qmpc_wse = np.load("./data/smpc_wse_qmpc.npy")
ax.stairs(range(len(qocp)-1), qocp, label="Stochastic OCP", color="blue", orientation="horizontal", baseline=None)
# ax.plot(qmpc[:-1], label="MPC control", color="orange")
ax.stairs(range(len(qmpc_wse[:-1])-1), qmpc_wse[:-1], label="MPCSE control", color="green", orientation="horizontal", baseline=None)
ax.legend()
ax.set_ylabel("Injection rate")
ax.set_xlabel("Time step")
ax.set_ylim([0.78, 1.22])
ax.grid()
plt.show()

socp_cost = np.load("./data/socp_cost_traj.npy")[:,:,0,0]
smpc_cost = np.load("./data/smpc_cost_traj.npy")[:,:,0,0]
smpc_cost_wse = np.load("./data/smpc_wse_cost_traj.npy")[:,:,0,0]

fig, ax = plt.subplots()
ax.plot(np.cumsum(socp_cost, axis=0), color='blue')
# ax.plot(np.cumsum(smpc_cost, axis=0), color='orange')
ax.plot(np.cumsum(smpc_cost_wse, axis=0), color='green')
ax.plot([], color='blue', label="Stochastic OCP")
# ax.plot([], color='orange', label="Stochastic MPC")
ax.plot([], color='green', label="Stochastic MPCSE")
ax.set_ylabel("NPV")
ax.set_xlabel("Time step")
ax.legend(loc="center right")

ax1 = ax.twinx()
ax1.plot(socp_cost, color="blue", linestyle="dashed")
# ax1.plot(smpc_cost, color="orange", linestyle="dashed")
ax1.plot(smpc_cost_wse, color="green", linestyle="dashed")
ax1.set_ylabel("Cashflow")

ax1.grid()



In [ ]:
npv_socp = np.cumsum(socp_cost, axis=0)[-1,:].mean()
npv_smpc = np.cumsum(smpc_cost, axis=0)[-1,:].mean()
npv_smpc_wse = np.cumsum(smpc_cost_wse, axis=0)[-1,:].mean()
npv_socp, npv_smpc, npv_smpc_wse

# Sanity check: calculate the cost using the same cost function for all the optimal controls

In [ ]:

from buckley_leverette_te import BuckleyLeverette
from setup_sbl_te import setup_sbl_ocp, BLParamsSmpcShort, BLParamsTrue

def calculate_cost(f_discrete, Swinit, qseq, aw, bw, Ne, nx):
    """Calculate cost function

    Args:
        Swinit (_type_): Initial Sw, shape (nx, Ne)
        qseq (_type_): Control sequence, shape (Nt)
        aw: a uncertainty, shape (nx, Ne)
        bw: b uncertainty, shape (nx, Ne)
    """
    
    Sw = Swinit*1
    
    cost_traj = []
    for k, q in enumerate(qseq):
        costs = []
        new_Sw = []
        for i in range(Ne):
            _Sw = Sw[i*nx:(i+1)*nx]
            cost = -ocp.bl._stage_cost(_Sw, q)*(0.99**k)
            # cost = -ocp.bl._stage_cost(_Sw, q)*(1.0**k)
            costs.append(cost.full()[0,0])
            
            _Sw = f_discrete(_Sw, q, aw[:,i], bw[:,i]).full()[:,0]
            new_Sw.extend(_Sw)
            
        cost_traj.append(costs)
        Sw = np.array(new_Sw)
        
    return np.array(cost_traj)
        
    
params_mpc = BLParamsSmpcShort(N_mpc = N_mpc, 
                                Ne = Ne,
                                nx = nx,
                                aw = aw,
                                bw = bw)

qocp = np.load("./data/socp_qocp.npy")
qmpc = [1.0]

ocp = setup_sbl_ocp(params_mpc, 
                    qmpc=qmpc,
                    qocp=qocp)
    

f_discrete = ocp.bl.simulate_at_k
    

# Stochastic OCP
_Sw0 = [1.0]+[0.2]*(params_mpc.nx-1)
Sw0 = np.array(_Sw0*Ne)  # initial state
qocp = np.load("./data/socp_qocp.npy")
aw = np.load("./data/aw.npy")
bw = np.load("./data/bw.npy")
socp_cost = calculate_cost(f_discrete, Sw0, qocp, aw, bw, Ne, nx)

# Stochastic MPC
_Sw0 = [1.0]+[0.2]*(params_mpc.nx-1)
Sw0 = np.array(_Sw0*Ne)  # initial state
qsmpc = np.load("./data/smpc_qmpc.npy")
aw = np.load("./data/aw.npy")
bw = np.load("./data/bw.npy")
smpc_cost = calculate_cost(f_discrete, Sw0, qsmpc, aw, bw, Ne, nx)

# Compute cost using the controls obtain from various method on the true model

In [ ]:

from buckley_leverette_te import BuckleyLeverette
from setup_sbl_te import setup_sbl_ocp, BLParamsSmpcShort, BLParamsTrue

def calculate_cost(f_discrete, Swinit, qseq, aw, bw, Ne, nx):
    """Calculate cost function

    Args:
        Swinit (_type_): Initial Sw, shape (nx, Ne)
        qseq (_type_): Control sequence, shape (Nt)
        aw: a uncertainty, shape (nx, Ne)
        bw: b uncertainty, shape (nx, Ne)
    """
    
    Sw = Swinit*1
    
    cost_traj = []
    for k, q in enumerate(qseq):
        costs = []
        new_Sw = []
        for i in range(Ne):
            _Sw = Sw[i*nx:(i+1)*nx]
            cost = -ocp.bl._stage_cost(_Sw, q)*(0.99**k)
            # cost = -ocp.bl._stage_cost(_Sw, q)*(1.0**k)
            costs.append(cost.full()[0,0])
            
            _Sw = f_discrete(_Sw, q, aw[:,i], bw[:,i]).full()[:,0]
            new_Sw.extend(_Sw)
            
        cost_traj.append(costs)
        Sw = np.array(new_Sw)
        
    return np.array(cost_traj)
        
    
## on true model
aw_true = np.load("./data/a_true.npy")
bw_true = np.load("./data/b_true.npy")

Ne = 1
nx = 25

params_mpc = BLParamsSmpcShort(N_mpc = N_mpc, 
                                Ne = Ne,
                                nx = nx,
                                aw = aw_true[:,np.newaxis],
                                bw = bw_true[:,np.newaxis])

qocp = np.load("./data/socp_qocp.npy")
qmpc = [1.0]

ocp = setup_sbl_ocp(params_mpc, 
                    qmpc=qmpc,
                    qocp=qocp)
    
f_discrete = ocp.bl.simulate_at_k
# Stochastic OCP
_Sw0 = [1.0]+[0.2]*(params_mpc.nx-1)
Sw0 = np.array(_Sw0*Ne)  # initial state
qocp = np.load("./data/socp_qocp.npy")
true_socp_cost = calculate_cost(f_discrete, Sw0, qocp, aw_true[:,np.newaxis], bw_true[:,np.newaxis], Ne, nx)
true_socp_cost_cum = np.cumsum(true_socp_cost, axis=0)

# Stochastic MPC
_Sw0 = [1.0]+[0.2]*(params_mpc.nx-1)
Sw0 = np.array(_Sw0*Ne)  # initial state
qsmpc = np.load("./data/smpc_qmpc.npy")
true_smpc_cost = calculate_cost(f_discrete, Sw0, qsmpc, aw_true[:,np.newaxis], bw_true[:,np.newaxis], Ne, nx)
true_smpc_cost_cum = np.cumsum(true_smpc_cost, axis=0)

# Stochastic MPC
_Sw0 = [1.0]+[0.2]*(params_mpc.nx-1)
Sw0 = np.array(_Sw0*Ne)  # initial state
qsmpc_wse = np.load("./data/smpc_wse_qmpc.npy")
true_smpc_wse_cost = calculate_cost(f_discrete, Sw0, qsmpc_wse, aw_true[:,np.newaxis], bw_true[:,np.newaxis], Ne, nx)
true_smpc_wse_cost_cum = np.cumsum(true_smpc_wse_cost, axis=0)

## on prior vs posterior model
aw_prior = np.load("./data/aw.npy")
bw_prior = np.load("./data/bw.npy")

Ne = 5
nx = 25

params_mpc = BLParamsSmpcShort(N_mpc = N_mpc, 
                                Ne = Ne,
                                nx = nx,
                                aw = aw_prior,
                                bw = bw_prior)

qocp = np.load("./data/socp_qocp.npy")
qmpc = [1.0]

ocp = setup_sbl_ocp(params_mpc, 
                    qmpc=qmpc,
                    qocp=qocp)
    
f_discrete = ocp.bl.simulate_at_k
# Prior model using old optimal control
_Sw0 = [1.0]+[0.2]*(params_mpc.nx-1)
Sw0 = np.array(_Sw0*Ne)  # initial state
qocp = np.load("./data/socp_qocp.npy")
prior_socp_cost = calculate_cost(f_discrete, Sw0, qocp, aw_prior, bw_prior, Ne, nx)
prior_socp_cost_cum = np.cumsum(prior_socp_cost, axis=0)

# Prior model using new optimal control
_Sw0 = [1.0]+[0.2]*(params_mpc.nx-1)
Sw0 = np.array(_Sw0*Ne)  # initial state
qsmpc_wse = np.load("./data/smpc_wse_qmpc.npy")
prior_smpc_wse_cost = calculate_cost(f_discrete, Sw0, qsmpc_wse, aw_prior, bw_prior, Ne, nx)
prior_smpc_wse_cost_cum = np.cumsum(prior_smpc_wse_cost, axis=0)[:-1,:]

aw_post = np.load("./data/aw_traj.npy")[-1,:,:]
bw_post = np.load("./data/bw_traj.npy")[-1,:,:]

params_mpc = BLParamsSmpcShort(N_mpc = N_mpc, 
                                Ne = Ne,
                                nx = nx,
                                aw = aw_post,
                                bw = aw_post)

qocp = np.load("./data/socp_qocp.npy")
qmpc = [1.0]

ocp = setup_sbl_ocp(params_mpc, 
                    qmpc=qmpc,
                    qocp=qocp)
    
f_discrete = ocp.bl.simulate_at_k
# Prior model using old optimal control
_Sw0 = [1.0]+[0.2]*(params_mpc.nx-1)
Sw0 = np.array(_Sw0*Ne)  # initial state
qocp = np.load("./data/socp_qocp.npy")
post_socp_cost = calculate_cost(f_discrete, Sw0, qocp, aw_post, bw_post, Ne, nx)
post_socp_cost_cum = np.cumsum(post_socp_cost, axis=0)

# Prior model using new optimal control
_Sw0 = [1.0]+[0.2]*(params_mpc.nx-1)
Sw0 = np.array(_Sw0*Ne)  # initial state
qsmpc_wse = np.load("./data/smpc_wse_qmpc.npy")
post_smpc_wse_cost = calculate_cost(f_discrete, Sw0, qsmpc_wse, aw_post, bw_post, Ne, nx)
post_smpc_wse_cost_cum = np.cumsum(post_smpc_wse_cost, axis=0)[:-1,:]


In [ ]:
data_prior = [prior_socp_cost_cum[-1,:], prior_smpc_wse_cost_cum[-1,:]]
data_post = [post_socp_cost_cum[-1,:], post_smpc_wse_cost_cum[-1,:]]
print(np.mean(prior_socp_cost_cum[-1,:]), np.mean(prior_smpc_wse_cost_cum[-1,:]))
fig, ax = plt.subplots(2,1,sharex=False)
fig.suptitle("Prior vs Posterior NPV")
ax[0].hist(data_prior, bins=5)
ax[1].hist(data_post, bins=5)
plt.show()

plt.figure()
data = [prior_socp_cost_cum[-1,:], post_smpc_wse_cost_cum[-1,:]]
plt.hist(data, bins=20)

# cost comparison of smpc vs socp control on true model

# data_true = [true_socp_cost_cum[-1,:], true_smpc_wse_cost_cum[-1,:]]
plt.figure()
plt.plot(true_socp_cost_cum)
plt.plot(true_smpc_wse_cost_cum)
plt.show()
print(f"Cumulative cost at the last time step: {true_socp_cost_cum[-1]} vs {true_smpc_wse_cost_cum[-2]} ")

## Prior vs posterior vs true parameters

In [ ]:
aw_traj = np.load("./data/aw_traj.npy")
bw_traj = np.load("./data/bw_traj.npy")

aw = np.load("./data/aw.npy")
bw = np.load("./data/bw.npy")

a_true = np.load("./data/a_true.npy")
b_true = np.load("./data/b_true.npy")

# plt.plot(aw_traj[1],color="grey")
# plt.plot(aw_traj[-1],color="blue")
# plt.plot(a_true[:],color="green")

fig, ax = plt.subplots(2, 1, figsize=(8,6))
ax[0].plot(aw, color='gray', alpha=0.5)
ax[0].plot(aw_traj[-1], color='blue', alpha=0.5)
ax[0].plot(a_true, color='red', linewidth=2)
ax[1].plot(bw, color='gray', alpha=0.5)
ax[1].plot(bw_traj[-1], color='blue', alpha=0.5)
ax[1].plot(b_true, color='red', linewidth=2)
ax[0].plot([], color='gray', alpha=0.5, label="a prior")
ax[0].plot([], color='blue', alpha=0.5, label="a posterior")
ax[0].plot([], color='red', linewidth=2, label="true a")
ax[1].plot([], color='gray', alpha=0.5, label="b prior")
ax[1].plot([], color='blue', alpha=0.5, label="b posterior")
ax[1].plot([], color='red', linewidth=2, label="true b")
ax[0].legend()
ax[1].legend()
plt.show()

## Distribution of the cost for prior vs posterior

